In [ ]:
from osgeo import osr, gdal
from openpyxl.workbook import Workbook

In [ ]:
crops = {} #number:cropname

In [ ]:
wb = Workbook()
sheetsdict = {'a': wb.active}
for nums, name in crops:
    sheetsdict[name]= wb.create_sheet()
    sheetsdict[name].title = name

In [ ]:
numexamples = {'a':0}
for nums, name in crops:
    numexamples[name] = 0
del numexamples['a']

In [ ]:
cd = gdal.Open('/Cropdata/Cropdata.tif')
cd_ncols = cd.RasterXSize
cd_nrows = cd.RasterYSize
cd_nbands = cd.RasterCount 
cd_gt = cd.GetGeoTransform()
cd_band = cd.GetRasterBand(1)
cd_data = cd_band.ReadAsArray(0, 0, width, height)
cd_cs= osr.SpatialReference()
cd_cs.ImportFromWkt(cd.GetProjectionRef())

In [ ]:
def Convert(i,j, sd_gt, sd_cs):
    transform = osr.CoordinateTransformation(sd_cs,cd_cs)
    o1 = sd_gt[0] + i *sd_gt[1] + j*sd_gt[2]
    o2 = sd_gt[3] + i *sd_gt[4] + j*sd_gt[5]
    return transform.TransformPoint(o1,o2)
    

In [ ]:
def Crop(i,j, sd_gt, sd_cs):
    q = Convert(i,j, sd_gt, sd_cs)
    x = int((q[0]-cd_gt[0])/gt[1])
    y = int((q[1]-cd_gt[3])/gt[5])
    try: 
        return data[y][x]
    except IndexError:
        return 0

In [ ]:
def YRN(i,j,sd_gt, sd_cs, crops):
    p = Crop(i,j,sd_gt, sd_cs)
    if p in crops:
        if Crop(i,j+1,sd_gt, sd_cs) == p:
            if Crop(i+1,j,sd_gt, sd_cs) == p:
                if Crop(i+1,j+1,sd_gt, sd_cs) == p:
                    return crops[p]
                else:
                    return 'USELESS'
            else:
                return 'USELESS'
        else:
            return 'USELESS'
    else:
        return 'USELESS'
    

In [ ]:
def processsat(file, crops = crops):
    croplists1 = {'a':[]}
    for numbers, names in crops:
        croplists1[names] = []
    del croplists1['a']
    croplists2 = {'a':[]}
    for numbers, names in crops:
        croplists2[names] = []
    del croplists2['a']
    croplists3 = {'a':[]}
    for numbers, names in crops:
        croplists3[names] = []
    del croplists3['a']
    croplists4 = {'a':[]}
    for numbers, names in crops:
        croplists4[names] = []
    del croplists4['a']
    sd = gdal.Open(file)
    sd_ncols = sd.RasterXSize
    sd_nrows = sd.RasterYSize
    sd_nbands = sd.RasterCount 
    sd_gt = sd.GetGeoTransform()
    sd_data1 = sd.GetRasterBand(1).ReadAsArray(0, 0, width, height)
    sd_data2 = sd.GetRasterBand(2).ReadAsArray(0, 0, width, height)
    sd_data3 = sd.GetRasterBand(3).ReadAsArray(0, 0, width, height)
    sd_data4 = sd.GetRasterBand(4).ReadAsArray(0, 0, width, height)
    sd_cs= osr.SpatialReference()
    sd_cs.ImportFromWkt(cd.GetProjectionRef())
    for i in range(sd_ncols):
        for j in range(sd_nrows):
            if sd_data1[j][i] == 0 and sd_data2[j][i] == 0 and sd_data3[j][i] == 0 and sd_data4[j][i] == 0:
                yrn = YRN(i,j, sd_gt, sd_cs, crops)
                if yrn != 'USELESS':
                    croplists1[yrn].append(sd_data1[j][i])
                    croplists2[yrn].append(sd_data2[j][i])
                    croplists3[yrn].append(sd_data3[j][i])
                    croplists4[yrn].append(sd_data4[j][i])
    return croplists1, croplists2, croplists3, croplists4

In [ ]:
def fillcell(croplists, columnnum):
    for crop, lst in croplists:
        r = numexamples[crop]
        sheet = sheetsdict[crop]
        for value in lst:
            r=r+1
            sheet.cell(row = r, column = columnnum).value = value
        if columnnum == 4:
            numexamples[crop] = r
        

In [ ]:
satfiles = ['/Satdata1/Satdata1/Satdata1.tif','/Satdata2/Satdata2/Satdata2.tif']
for file in satfiles:
    croplists1,croplists2, croplists3, croplists4 = processsat(file) 
    columnnum = 1
    for croplists in [croplists1,croplists2, croplists3, croplists4]:
        fillcell(croplists = croplists, columnnum = columnnum)
        columnnum = columnnum + 1

In [ ]:
wb.save('labelled_data.xlsx')